In [6]:
#@title Installation
from importlib.util import find_spec
in_colab = find_spec("google.colab") is not None


src = """[tool.uv]
override-dependencies = ["""
replace_with = """
    "torch ; sys_platform == 'never'",
    "pytorch-triton ; sys_platform == 'never'",
"""

if in_colab:
    %env SPARSIFY_DISABLE_TRITON=1
    %env OFFLOAD_TRANSCODER=1
    %cd /content
    !git clone https://github.com/EleutherAI/attribute
    %cd attribute
    !git pull
    pyproject_text = open("pyproject.toml").read()
    if src in pyproject_text:
        after_src = pyproject_text.index(src) + len(src)
        current = pyproject_text[after_src:][:len(replace_with)]
        if current != replace_with:
            pyproject_text = pyproject_text[:after_src] + replace_with + pyproject_text[after_src:]
            open("pyproject.toml", "w").write(pyproject_text)
    !curl -LsSf https://astral.sh/uv/install.sh | sh
    !uv pip install -e .
    !pip install triton

env: SPARSIFY_DISABLE_TRITON=1
env: OFFLOAD_TRANSCODER=1
/content
fatal: destination path 'attribute' already exists and is not an empty directory.
/content/attribute
Already up to date.
downloading uv 0.7.13 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!
Using Python 3.11.13 environment at: /usr
Resolved 161 packages in 227ms
   Building attribution-graph @ file:///content/attribute
   Building attribution-graph @ file:///content/attribute
   Building attribution-graph @ file:///content/attribute
   Building attribution-graph @ file:///content/attribute
      Built attribution-graph @ file:///content/attribute
Prepared 1 package in 671ms
Uninstalled 1 package in 0.55ms
Installed 1 package in 0.89ms
 ~ attribution-graph==0.1.0 (from file:///content/attribute)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 7.3 MB/s eta 0:00:00


In [2]:
import huggingface_hub
huggingface_hub.login()

In [3]:
#@markdown this flag is necesssary viewing feature visualizations. however, cache files take a while to download and process, so you may want to disable this.
download_llama_cache = True  #@param {type:"boolean"}
if download_llama_cache:
    !huggingface-cli download EleutherAI/Llama-3.2-1B-mntss-skip-transcoder-cache-1M --local-dir results/transcoder-llama-131k-mntss
download_gemma_cache = False  #@param {type:"boolean"}
if download_gemma_cache:
    !huggingface-cli download EleutherAI/gemmascope-transcoders-sparsify-cache-1m --local-dir results/gemmascope-transcoders-sparsify-1m
download_cache = download_llama_cache or download_gemma_cache

Fetching 98 files:   0% 0/98 [00:00<?, ?it/s]Downloading '.gitattributes' to 'results/transcoder-llama-131k-mntss/.cache/huggingface/download/wPaCkH-WbT7GsmxMKKrNZTV4nSM=.a6344aac8c09253b3b630fb776ae94478aa0275b.incomplete'

config.json: 100% 279/279 [00:00<00:00, 2.70MB/s]
Download complete. Moving file to results/transcoder-llama-131k-mntss/latents/layers.0.mlp/config.json

.gitattributes: 100% 1.52k/1.52k [00:00<00:00, 16.3MB/s]
Download complete. Moving file to results/transcoder-llama-131k-mntss/.gitattributes
Fetching 98 files:   1% 1/98 [00:00<00:22,  4.35it/s]
0_26213.safetensors:   0% 0.00/107M [00:00<?, ?B/s]Downloading 'latents/layers.1.mlp/26214_52427.safetensors' to 'results/transcoder-llama-131k-mntss/.cache/huggingface/download/latents/layers.1.mlp/DfI343hiqeb5QZh_dzGOa9U1tOI=.d9893fa5fba58f05ce62561a340bbb3d21a35fb96e590113911d97e58c2d5851.incomplete'

0_26213.safetensors:  10% 10.5M/107M [00:00<00:01, 59.0MB/s]

52428_78642.safetensors:   0% 0.00/106M [00:00<?, ?B/s]



In [7]:
import serve
serve.default_config.batch_size = 8
serve.default_config.flow_steps = 1000
if not download_cache:
    serve.default_config.use_self_explanation = True
serve.main().launch(share=True, debug=True)

Triton disabled, using eager implementation of sparse decoder.
['attribution-graphs-frontend', 'attribution-graphs-frontend/features/transcoder-llama-131k-mntss', 'attribution-graphs-frontend/features/gemmascope-transcoders-sparsify', 'attribution-graphs-frontend/features/smollm-v1']
Caching examples at: '/content/attribute/.gradio/cached_examples/23'
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8c00ed3685ed34e96c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


2025-06-16 20:26:11.540 | INFO     | attribute.caching:__init__:78 - Loading model meta-llama/Llama-3.2-1B on device cuda
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_http.py", line 409, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/requests/models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPErro

config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2220, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1731, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2220, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1731, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8c00ed3685ed34e96c.gradio.live
